In [12]:
!pip install pyomo -q
!pip install mistralai -q
!wget -N -q "https://matematica.unipv.it/gualandi/solvers/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64

In [13]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from IPython.display import display, Markdown, Latex
from datetime import datetime

In [14]:
API_KEY = ''

MODEL_ID = 'open-mixtral-8x22b'
MODEL_SEED = 1
MODEL_TEMPERATURE = 0.7

SYSTEM_PROMPT_1 = """Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."""
SYSTEM_PROMPT_2 = """Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."""
PROBLEM_DESCRIPTION = """We are looking at an alkylation process which will include the following 10 variables: olefin feed (barrels per day), isobutane recycle (barrels per day), acid addition rate (thousands of pounds per day), alkylate yield (barrels per day), isobutane makeup (barrels per day), acid strength (weight per cent), motor octane number, external isobutane-to-olefin ratio, acid dilution factor and F-4 performance number.

We want to maximize the daily profit of this alkylation process.
The profit is defined as the revenue generated from the alkylate yield multiplied with the motor octane number, minus the operational costs, which include olefin feed, isobutane recycle, acid addition rate, and isobutane makeup.

Relationships in terms of other variables for alkylate yield, motor octane number, acid dilution factor, and F-4 performance number can be formulated as regression formulas.
This regression estimate can deviate in both directions from true value of these variables by 2, 1, 5 and 10 percent, respectively.
Alkylate yield is a function of olefin feed and external isobutane-to-olefine yield. Alkalyte yield equals the amount of olefin feed multiplied by the sum of 1.12, 0.13167 times the external isobutane-to-olefin ratio and -0.00667 times the external isobutane-to-olefin ratio squared.
The motor octane number is derived from the external isobutane-to-olefin ratio and the acid strength. The motor octane number is calculated as the sum of 86.35, 1.098 time external isobutane-to-olefin ratio, -0.038 times the external isobutane-to-olefin ratio squared and 0.325 times acid strength reduced by 89.
The acid dilution factor is calculated based on the F-4 performance number. The acid dillution factor is expressed as 35.82 minus 0.222 times F-4 performance number.
Lastly, the F-4 performance number depends on the motor octane number. F-4 performance number is calculated as -133 plus three times the motor octane number.

There are some additional constraints imposed by the nature of the chemical process.
Each variable has a lower and an upper bound.
The external isobutane-to-olefin ratio needs to equal the ratio of isobutane recycle plus isobutane makeup to olefin feed.
The acid strength needs to equal the ratio of 93000 times acid addition rate to acid addition rate multiplied by acid dilution factor in addition to 1000 times acid addition rate.
Lastly, 1.22 alkylate yield needs to be equal to the combined olefin feed and isobutane makeup."""


client = MistralClient(api_key=API_KEY)

print(f'Time of execution: {datetime.now()}')

Time of execution: 2024-06-14 15:06:06.260021


## Step 1 - Generate Mathematical Formulation

In [15]:
messages_1 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION)
]

In [16]:
response_1 = client.chat(
    model=MODEL_ID,
    messages=messages_1,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_1_text = response_1.choices[0].message.content

In [17]:
Markdown(response_1_text)

Here is a mathematical optimization model for the problem:

Parameters:

* $c_{alkylate}$: Revenue per barrel of alkylate (constant)
* $c_{olefin}$: Cost per barrel of olefin feed (constant)
* $c_{isobutane\_recycle}$: Cost per barrel of isobutane recycle (constant)
* $c_{acid}$: Cost per thousand pounds of acid addition (constant)
* $c_{isobutane\_makeup}$: Cost per barrel of isobutane makeup (constant)

Decision Variables:

* $x_{olefin}$: Olefin feed (barrels per day)
* $x_{isobutane\_recycle}$: Isobutane recycle (barrels per day)
* $x_{acid\_addition}$: Acid addition rate (thousands of pounds per day)
* $x_{isobutane\_makeup}$: Isobutane makeup (barrels per day)
* $x_{acid\_strength}$: Acid strength (weight per cent)
* $x_{external\_isobutane\_to\_olefin\_ratio}$: External isobutane-to-olefin ratio
* $x_{acid\_dilution\_factor}$: Acid dilution factor
* $x_{F\_4\_performance\_number}$: F-4 performance number

Objective Function:

Maximize $c_{alkylate} \cdot x_{alkylate\_yield} \cdot x_{motor\_octane\_number} - c_{olefin} \cdot x_{olefin} - c_{isobutane\_recycle} \cdot x_{isobutane\_recycle} - c_{acid} \cdot x_{acid\_addition} - c_{isobutane\_makeup} \cdot x_{isobutane\_makeup}$

Constraints:

1. Alkylate yield: $x_{alkylate\_yield} = x_{olefin} \cdot (1.12 + 0.13167 \cdot x_{external\_isobutane\_to\_olefin\_ratio} - 0.00667 \cdot x_{external\_isobutane\_to\_olefin\_ratio}^2)$
2. Motor octane number: $x_{motor\_octane\_number} = 86.35 + 1.098 \cdot x_{external\_isobutane\_to\_olefin\_ratio} - 0.038 \cdot x_{external\_isobutane\_to\_olefin\_ratio}^2 + 0.325 \cdot (x_{acid\_strength} - 89)$
3. Acid dilution factor: $x_{acid\_dilution\_factor} = 35.82 - 0.222 \cdot x_{F\_4\_performance\_number}$
4. F-4 performance number: $x_{F\_4\_performance\_number} = -133 + 3 \cdot x_{motor\_octane\_number}$
5. External isobutane-to-olefin ratio: $x_{external\_isobutane\_to\_olefin\_ratio} = \frac{x_{isobutane\_recycle} + x_{isobutane\_makeup}}{x_{olefin}}$
6. Acid strength: $x_{acid\_strength} = \frac{93000 \cdot x_{acid\_addition}}{x_{acid\_addition} \cdot x_{acid\_dilution\_factor} + 1000 \cdot x_{acid\_addition}}$
7. Alkylate yield constraint: $1.22 \cdot x_{alkylate\_yield} = x_{olefin} + x_{isobutane\_makeup}$
8. Bounds on decision variables:
	* $0 \leq x_{olefin} \leq x_{olefin\_max}$
	* $0 \leq x_{isobutane\_recycle} \leq x_{isobutane\_recycle\_max}$
	* $0 \leq x_{acid\_addition} \leq x_{acid\_addition\_max}$
	* $0 \leq x_{isobutane\_makeup} \leq x_{isobutane\_makeup\_max}$
	* $0 \leq x_{acid\_strength} \leq x_{acid\_strength\_max}$
	* $x_{external\_isobutane\_to\_olefin\_ratio\_min} \leq x_{external\_isobutane\_to\_olefin\_ratio} \leq x_{external\_isobutane\_to\_olefin\_ratio\_max}$
	* $x_{acid\_dilution\_factor\_min} \leq x_{acid\_dilution\_factor} \leq x_{acid\_dilution\_factor\_max}$
	* $x_{F\_4\_performance\_number\_min} \leq x_{F\_4\_performance\_number} \leq x_{F\_4\_performance\_number\_max}$

In this model, the objective function maximizes the daily profit, which is the revenue generated from alkylate yield multiplied with the motor octane number, minus the operational costs. The constraints represent the relationships between the variables and the bounds on the variables. The regression estimates for alkylate yield, motor octane number, acid dilution factor, and F-4 performance number are incorporated into the constraints. The bounds on the variables are also included as constraints.

In [18]:
print(response_1_text)

Here is a mathematical optimization model for the problem:

Parameters:

* $c_{alkylate}$: Revenue per barrel of alkylate (constant)
* $c_{olefin}$: Cost per barrel of olefin feed (constant)
* $c_{isobutane\_recycle}$: Cost per barrel of isobutane recycle (constant)
* $c_{acid}$: Cost per thousand pounds of acid addition (constant)
* $c_{isobutane\_makeup}$: Cost per barrel of isobutane makeup (constant)

Decision Variables:

* $x_{olefin}$: Olefin feed (barrels per day)
* $x_{isobutane\_recycle}$: Isobutane recycle (barrels per day)
* $x_{acid\_addition}$: Acid addition rate (thousands of pounds per day)
* $x_{isobutane\_makeup}$: Isobutane makeup (barrels per day)
* $x_{acid\_strength}$: Acid strength (weight per cent)
* $x_{external\_isobutane\_to\_olefin\_ratio}$: External isobutane-to-olefin ratio
* $x_{acid\_dilution\_factor}$: Acid dilution factor
* $x_{F\_4\_performance\_number}$: F-4 performance number

Objective Function:

Maximize $c_{alkylate} \cdot x_{alkylate\_yield} \cdo

## Step 2 - Generate the Pyomo Code

In [19]:
messages_2 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_2),
    ChatMessage(role="user", content=response_1_text)
]

In [20]:
response_2 = client.chat(
    model=MODEL_ID,
    messages=messages_2,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_2_text = response_2.choices[0].message.content

In [21]:
Markdown(response_2_text)

Here is a Python code using Pyomo to solve the optimization problem:

```python
import pyomo.environ as pyo

# Define the model
model = pyo.ConcreteModel()

# Define the parameters
model.c_alkylate = pyo.Param(initialize=50)  # Sample data: Revenue per barrel of alkylate
model.c_olefin = pyo.Param(initialize=10)  # Sample data: Cost per barrel of olefin feed
model.c_isobutane_recycle = pyo.Param(initialize=5)  # Sample data: Cost per barrel of isobutane recycle
model.c_acid = pyo.Param(initialize=20)  # Sample data: Cost per thousand pounds of acid addition
model.c_isobutane_makeup = pyo.Param(initialize=7)  # Sample data: Cost per barrel of isobutane makeup

# Define the decision variables
model.x_olefin = pyo.Var(within=pyo.NonNegativeReals, bounds=(0, 10000))  # Olefin feed (barrels per day)
model.x_isobutane_recycle = pyo.Var(within=pyo.NonNegativeReals, bounds=(0, 10000))  # Isobutane recycle (barrels per day)
model.x_acid_addition = pyo.Var(within=pyo.NonNegativeReals, bounds=(0, 1000))  # Acid addition rate (thousands of pounds per day)
model.x_isobutane_makeup = pyo.Var(within=pyo.NonNegativeReals, bounds=(0, 10000))  # Isobutane makeup (barrels per day)
model.x_acid_strength = pyo.Var(within=pyo.NonNegativeReals, bounds=(0, 100))  # Acid strength (weight per cent)
model.x_external_isobutane_to_olefin_ratio = pyo.Var(within=pyo.NonNegativeReals)  # External isobutane-to-olefin ratio
model.x_acid_dilution_factor = pyo.Var(within=pyo.NonNegativeReals)  # Acid dilution factor
model.x_F_4_performance_number = pyo.Var(within=pyo.NonNegativeReals)  # F-4 performance number

# Define the objective function
def obj_expression(model):
    return (
        model.c_alkylate * model.x_alkylate_yield * model.x_motor_octane_number
        - model.c_olefin * model.x_olefin
        - model.c_isobutane_recycle * model.x_isobutane_recycle
        - model.c_acid * model.x_acid_addition
        - model.c_isobutane_makeup * model.x_isobutane_makeup
    )

model.obj = pyo.Objective(rule=obj_expression, sense=pyo.maximize)

# Define the constraints
def alkylate_yield_constraint_rule(model):
    return model.x_alkylate_yield == (
        model.x_olefin * (1.12 + 0.13167 * model.x_external_isobutane_to_olefin_ratio - 0.00667 * model.x_external_isobutane_to_olefin_ratio**2)
    )

model.alkylate_yield_constraint = pyo.Constraint(rule=alkylate_yield_constraint_rule)

def motor_octane_number_constraint_rule(model):
    return model.x_motor_octane_number == (
        86.35 + 1.098 * model.x_external_isobutane_to_olefin_ratio - 0.038 * model.x_external_isobutane_to_olefin_ratio**2
        + 0.325 * (model.x_acid_strength - 89)
    )

model.motor_octane_number_constraint = pyo.Constraint(rule=motor_octane_number_constraint_rule)

def acid_dilution_factor_constraint_rule(model):
    return model.x_acid_dilution_factor == (35.82 - 0.222 * model.x_F_4_performance_number)

model.acid_dilution_factor_constraint = pyo.Constraint(rule=acid_dilution_factor_constraint_rule)

def F_4_performance_number_constraint_rule(model):
    return model.x_F_4_performance_number == (-133 + 3 * model.x_motor_octane_number)

model.F_4_performance_number_constraint = pyo.Constraint(rule=F_4_performance_number_constraint_rule)

def external_isobutane_to_olefin_ratio_constraint_rule(model):
    return model.x_external_isobutane_to_olefin_ratio == (model.x_isobutane_recycle + model.x_isobutane_makeup) / model.x_olefin

model.external_isobutane_to_olefin_ratio_constraint = pyo.Constraint(rule=external_isobutane_to_olefin_ratio_constraint_rule)

def acid_strength_constraint_rule(model):
    return model.x_acid_strength == (93000 * model.x_acid_addition) / (model.x_acid_addition * model.x_acid_dilution_factor + 1000 * model.x_acid_addition)

model.acid_strength_constraint = pyo.Constraint(rule=acid_strength_constraint_rule)

def alkylate_yield_bound_constraint_rule(model):
    return 1.22 * model.x_alkylate_yield == model.x_olefin + model.x_isobutane_makeup

model.alkylate_yield_bound_constraint = pyo.Constraint(rule=alkylate_yield_bound_constraint_rule)

# Solve the model
solver = pyo.SolverFactory('ipopt')  # Replace 'ipopt' with the desired solver
solver.solve(model)

# Print the results
print("Optimal Solution:")
print("Objective Function Value: ", pyo.value(model.obj))
print("Olefin Feed: ", pyo.value(model.x_olefin))
print("Isobutane Recycle: ", pyo.value(model.x_isobutane_recycle))
print("Acid Addition Rate: ", pyo.value(model.x_acid_addition))
print("Isobutane Makeup: ", pyo.value(model.x_isobutane_makeup))
print("Acid Strength: ", pyo.value(model.x_acid_strength))
print("External Isobutane-to-Olefin Ratio: ", pyo.value(model.x_external_isobutane_to_olefin_ratio))
print("Acid Dilution Factor: ", pyo.value(model.x_acid_dilution_factor))
print("F-4 Performance Number: ", pyo.value(model.x_F_4_performance_number))
```

Replace the sample data with the actual values for the parameters and adjust the bounds on the decision variables as needed. The solver used in this code is 'ipopt', but you can replace it with the desired solver.

In [22]:
print(response_2_text)

Here is a Python code using Pyomo to solve the optimization problem:

```python
import pyomo.environ as pyo

# Define the model
model = pyo.ConcreteModel()

# Define the parameters
model.c_alkylate = pyo.Param(initialize=50)  # Sample data: Revenue per barrel of alkylate
model.c_olefin = pyo.Param(initialize=10)  # Sample data: Cost per barrel of olefin feed
model.c_isobutane_recycle = pyo.Param(initialize=5)  # Sample data: Cost per barrel of isobutane recycle
model.c_acid = pyo.Param(initialize=20)  # Sample data: Cost per thousand pounds of acid addition
model.c_isobutane_makeup = pyo.Param(initialize=7)  # Sample data: Cost per barrel of isobutane makeup

# Define the decision variables
model.x_olefin = pyo.Var(within=pyo.NonNegativeReals, bounds=(0, 10000))  # Olefin feed (barrels per day)
model.x_isobutane_recycle = pyo.Var(within=pyo.NonNegativeReals, bounds=(0, 10000))  # Isobutane recycle (barrels per day)
model.x_acid_addition = pyo.Var(within=pyo.NonNegativeReals, bounds=(0,

### Code Executability

In [23]:
import pyomo.environ as pyo

# Define the model
model = pyo.ConcreteModel()

# Define the parameters
model.c_alkylate = pyo.Param(initialize=50)  # Sample data: Revenue per barrel of alkylate
model.c_olefin = pyo.Param(initialize=10)  # Sample data: Cost per barrel of olefin feed
model.c_isobutane_recycle = pyo.Param(initialize=5)  # Sample data: Cost per barrel of isobutane recycle
model.c_acid = pyo.Param(initialize=20)  # Sample data: Cost per thousand pounds of acid addition
model.c_isobutane_makeup = pyo.Param(initialize=7)  # Sample data: Cost per barrel of isobutane makeup

# Define the decision variables
model.x_olefin = pyo.Var(within=pyo.NonNegativeReals, bounds=(0, 10000))  # Olefin feed (barrels per day)
model.x_isobutane_recycle = pyo.Var(within=pyo.NonNegativeReals, bounds=(0, 10000))  # Isobutane recycle (barrels per day)
model.x_acid_addition = pyo.Var(within=pyo.NonNegativeReals, bounds=(0, 1000))  # Acid addition rate (thousands of pounds per day)
model.x_isobutane_makeup = pyo.Var(within=pyo.NonNegativeReals, bounds=(0, 10000))  # Isobutane makeup (barrels per day)
model.x_acid_strength = pyo.Var(within=pyo.NonNegativeReals, bounds=(0, 100))  # Acid strength (weight per cent)
model.x_external_isobutane_to_olefin_ratio = pyo.Var(within=pyo.NonNegativeReals)  # External isobutane-to-olefin ratio
model.x_acid_dilution_factor = pyo.Var(within=pyo.NonNegativeReals)  # Acid dilution factor
model.x_F_4_performance_number = pyo.Var(within=pyo.NonNegativeReals)  # F-4 performance number

# Define the objective function
def obj_expression(model):
    return (
        model.c_alkylate * model.x_alkylate_yield * model.x_motor_octane_number
        - model.c_olefin * model.x_olefin
        - model.c_isobutane_recycle * model.x_isobutane_recycle
        - model.c_acid * model.x_acid_addition
        - model.c_isobutane_makeup * model.x_isobutane_makeup
    )

model.obj = pyo.Objective(rule=obj_expression, sense=pyo.maximize)

# Define the constraints
def alkylate_yield_constraint_rule(model):
    return model.x_alkylate_yield == (
        model.x_olefin * (1.12 + 0.13167 * model.x_external_isobutane_to_olefin_ratio - 0.00667 * model.x_external_isobutane_to_olefin_ratio**2)
    )

model.alkylate_yield_constraint = pyo.Constraint(rule=alkylate_yield_constraint_rule)

def motor_octane_number_constraint_rule(model):
    return model.x_motor_octane_number == (
        86.35 + 1.098 * model.x_external_isobutane_to_olefin_ratio - 0.038 * model.x_external_isobutane_to_olefin_ratio**2
        + 0.325 * (model.x_acid_strength - 89)
    )

model.motor_octane_number_constraint = pyo.Constraint(rule=motor_octane_number_constraint_rule)

def acid_dilution_factor_constraint_rule(model):
    return model.x_acid_dilution_factor == (35.82 - 0.222 * model.x_F_4_performance_number)

model.acid_dilution_factor_constraint = pyo.Constraint(rule=acid_dilution_factor_constraint_rule)

def F_4_performance_number_constraint_rule(model):
    return model.x_F_4_performance_number == (-133 + 3 * model.x_motor_octane_number)

model.F_4_performance_number_constraint = pyo.Constraint(rule=F_4_performance_number_constraint_rule)

def external_isobutane_to_olefin_ratio_constraint_rule(model):
    return model.x_external_isobutane_to_olefin_ratio == (model.x_isobutane_recycle + model.x_isobutane_makeup) / model.x_olefin

model.external_isobutane_to_olefin_ratio_constraint = pyo.Constraint(rule=external_isobutane_to_olefin_ratio_constraint_rule)

def acid_strength_constraint_rule(model):
    return model.x_acid_strength == (93000 * model.x_acid_addition) / (model.x_acid_addition * model.x_acid_dilution_factor + 1000 * model.x_acid_addition)

model.acid_strength_constraint = pyo.Constraint(rule=acid_strength_constraint_rule)

def alkylate_yield_bound_constraint_rule(model):
    return 1.22 * model.x_alkylate_yield == model.x_olefin + model.x_isobutane_makeup

model.alkylate_yield_bound_constraint = pyo.Constraint(rule=alkylate_yield_bound_constraint_rule)

# Solve the model
solver = pyo.SolverFactory('ipopt')  # Replace 'ipopt' with the desired solver
solver.solve(model)

# Print the results
print("Optimal Solution:")
print("Objective Function Value: ", pyo.value(model.obj))
print("Olefin Feed: ", pyo.value(model.x_olefin))
print("Isobutane Recycle: ", pyo.value(model.x_isobutane_recycle))
print("Acid Addition Rate: ", pyo.value(model.x_acid_addition))
print("Isobutane Makeup: ", pyo.value(model.x_isobutane_makeup))
print("Acid Strength: ", pyo.value(model.x_acid_strength))
print("External Isobutane-to-Olefin Ratio: ", pyo.value(model.x_external_isobutane_to_olefin_ratio))
print("Acid Dilution Factor: ", pyo.value(model.x_acid_dilution_factor))
print("F-4 Performance Number: ", pyo.value(model.x_F_4_performance_number))

ERROR:pyomo.core:Rule failed when generating expression for Objective obj with index None:
AttributeError: 'ConcreteModel' object has no attribute 'x_alkylate_yield'
ERROR:pyomo.core:Constructing component 'obj' from data=None failed:
    AttributeError: 'ConcreteModel' object has no attribute 'x_alkylate_yield'


AttributeError: 'ConcreteModel' object has no attribute 'x_alkylate_yield'

### Solution Correctness